In [11]:
%%capture
!pip install -qU langchain huggingface_hub lancedb pypdf python-dotenv transformers sentence-transformers

In [99]:
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain.llms import OpenAI
import lancedb

In [100]:
import os
from getpass import getpass
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter HuggingFace Hub Token:")

Enter HuggingFace Hub Token: ········


In [101]:
loader = PyPDFLoader("Science_Glossary.pdf")
documents = loader.load()
print(len(documents))
print(documents[0].page_content)

7
SCIENCE GLOSSARY 
 
Abiotic:   A nonliving factor or element (e.g., light, w ater, heat, rock, energy, mineral). 
 
Acid deposition:  Precipitation w ith a pH less than 5.6 that form s in the atmosphere w hen certain pollutants mix 
with water vapor. 
 
Allele:   Any of a set of possible forms of a gene. 
 
Biochemical conversion:  The changing of organic matter into other chemical forms. 
 
Biological diversity:  The variety and complexity of species present a nd interacting in an ecosystem and the relative 
abundance of each. 
 
Biomass conversion: The changing of organic matter that has been produced by photosynthesis into useful liquid, gas 
or fuel. 
 
Biomedical technology: The application of health care theories  to develop methods, products and tools to maintain or 
improve homeostasis. 
 
Biomes:   A community of living organisms of a single major ecological region. 
 
Biotechnology:   The w ays that humans apply biological concepts  to produce products and provide services.

In [102]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700,chunk_overlap=70)
final_doc = text_splitter.split_documents(documents)
print(len(final_doc))
print(final_doc[0])

22
page_content='SCIENCE GLOSSARY \n \nAbiotic:   A nonliving factor or element (e.g., light, w ater, heat, rock, energy, mineral). \n \nAcid deposition:  Precipitation w ith a pH less than 5.6 that form s in the atmosphere w hen certain pollutants mix \nwith water vapor. \n \nAllele:   Any of a set of possible forms of a gene. \n \nBiochemical conversion:  The changing of organic matter into other chemical forms. \n \nBiological diversity:  The variety and complexity of species present a nd interacting in an ecosystem and the relative \nabundance of each. \n \nBiomass conversion: The changing of organic matter that has been produced by photosynthesis into useful liquid, gas \nor fuel.' metadata={'source': 'Science_Glossary.pdf', 'page': 0}


In [103]:
embeddings = SentenceTransformerEmbeddings(model_name="llmware/industry-bert-insurance-v0.1")

No sentence-transformers model found with name C:\Users\Lenovo/.cache\torch\sentence_transformers\llmware_industry-bert-insurance-v0.1. Creating a new one with MEAN pooling.


In [104]:
repo_id ="llmware/bling-sheared-llama-1.3b-0.1"
llm = HuggingFaceHub(repo_id=repo_id,
                     model_kwargs={"temperature":0.3,"max_length":500})

C:\Users\Lenovo\Documents\workspace\lim\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [105]:
def pretty_print_docs(docs):
  print(f"\n{'-'* 100}\n".join([F"Document{i+1}:\n\n" + d.page_content for i,d in enumerate(docs)]))

In [106]:
import lancedb
context_data = lancedb.connect("./.lancedb")
table = context_data.create_table("context", data=[
    {"vector": embeddings.embed_query("Hello World"), "text": "Hello World", "id": "1"}
], mode="overwrite")

In [107]:
#initialize the retriever
database = LanceDB.from_documents(final_doc, embeddings, connection=table)

In [108]:
retriever_d = database.as_retriever(search_kwargs={"k": 3})

In [109]:
docs = retriever_d.get_relevant_documents(query="What is Wetlands?")
pretty_print_docs(docs)

Document1:

body of w ater; also called a drainage basin. 
  
Wetlands: Lands w here w ater saturation is the domi nant factor determining the nature of the soil 
development and the plant and animal communities (e.g., sloughs, estuaries, marshes). 
 
 
 7
----------------------------------------------------------------------------------------------------
Document2:

developed state. 
 
Endangered species:   A species that is in danger of extinction throughout all or a significant portion of its range. 
 
Engineering:  The application of scientific, physical, mechan ical and mathematical principles to design 
processes, products and structures that improve the quality of life. 
 
Environment:  The total of the surroundings (air, w ater, soil, vege tation, people, w ildlife) influencing each living 
being’s existence, including physical, biological a nd all other factors; the surroundings of a plant 
or animals including other plants or animals, climate and location. 
 2
---------------

In [110]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

#creating the compressor
compressor = LLMChainExtractor.from_llm(llm=llm)

#compressor retriver = base retriever + compressor
compression_retriever = ContextualCompressionRetriever(base_retriever=retriever_d,
                                                       base_compressor=compressor)

In [111]:
print(compressor.llm_chain.prompt.template)

Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. 

Remember, *DO NOT* edit the extracted parts of the context.

> Question: {question}
> Context:
>>>
{context}
>>>
Extracted relevant parts:


In [112]:
from getpass import getpass
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.document_compressors import EmbeddingsFilter

os.environ["OPENAI_API_KEY "] = getpass()
embdeddings_filter = EmbeddingsFilter(embeddings=embeddings)
compression_retriever_filter = ContextualCompressionRetriever(base_retriever=retriever_d,
                                                       base_compressor=embdeddings_filter)

compressed_docs = compression_retriever_filter.get_relevant_documents(query="What is the Environment?")
pretty_print_docs(compressed_docs)

 ········


Document1:

developed state. 
 
Endangered species:   A species that is in danger of extinction throughout all or a significant portion of its range. 
 
Engineering:  The application of scientific, physical, mechan ical and mathematical principles to design 
processes, products and structures that improve the quality of life. 
 
Environment:  The total of the surroundings (air, w ater, soil, vege tation, people, w ildlife) influencing each living 
being’s existence, including physical, biological a nd all other factors; the surroundings of a plant 
or animals including other plants or animals, climate and location. 
 2
----------------------------------------------------------------------------------------------------
Document2:

Niche (ecological) : The role played by an organism in an ecosystem; its food preferences, requirements for shelter, 
special behaviors and the timing of its activities  (e.g., nocturnal, diurnal), interaction w ith other 
organisms and its habitat. 
 
Nonpoin

In [113]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever_filter,
                                 verbose=True)
#Ask Question
qa("What is Environment?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is Environment?', 'result': ' Environment'}

# Pipeline

In [114]:
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline

redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings,k=5)

#creating the pipeline
compressed_pipeline = DocumentCompressorPipeline(transformers=[redundant_filter,relevant_filter])

# compressor retriever
comp_pipe_retrieve = ContextualCompressionRetriever(base_retriever=retriever_d,
                                                       base_compressor=compressed_pipeline)

# print the prompt
print(comp_pipe_retrieve)

# Get relevant documents
compressed_docs = comp_pipe_retrieve.get_relevant_documents(query="What is Environment?")
pretty_print_docs(compressed_docs)

base_compressor=DocumentCompressorPipeline(transformers=[EmbeddingsRedundantFilter(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='llmware/industry-bert-insurance-v0.1', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False), similarity_fn=<function cosine_similarity at 0x000001CE19480F40>, similarity_threshold=0.95), EmbeddingsFilter(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'po

In [115]:
compressed_docs = compression_retriever_pipeline.get_relevant_documents(query="What is Environment?")
pretty_print_docs(compressed_docs)

Document1:

developed state. 
 
Endangered species:   A species that is in danger of extinction throughout all or a significant portion of its range. 
 
Engineering:  The application of scientific, physical, mechan ical and mathematical principles to design 
processes, products and structures that improve the quality of life. 
 
Environment:  The total of the surroundings (air, w ater, soil, vege tation, people, w ildlife) influencing each living 
being’s existence, including physical, biological a nd all other factors; the surroundings of a plant 
or animals including other plants or animals, climate and location. 
 2
----------------------------------------------------------------------------------------------------
Document2:

Niche (ecological) : The role played by an organism in an ecosystem; its food preferences, requirements for shelter, 
special behaviors and the timing of its activities  (e.g., nocturnal, diurnal), interaction w ith other 
organisms and its habitat. 
 
Nonpoin

In [116]:
compressed_docs = compression_retriever_pipeline.get_relevant_documents(query="What is Hazardous waste?")
pretty_print_docs(compressed_docs)

Document1:

and age relationships of rock units and the occu rrences of structural features, mineral deposits 
and fossil localities).  
 
Groundw ater:   Water that infiltrates the soil and is located in underground reservoirs called aquifers. 
 
Hazardous w aste: A solid that, because of its quantity or concentr ation or its physical, chemical or infectious 
characteristics, may cause or pose a substantial present or potential haz ard to human health or 
the environment w hen improperly treated, stored , transported or disposed of, or otherw ise 
managed. 
 
Homeostasis:   The tendency for a system to remain in a state of equilibrium by resisting change. 
 
 3
----------------------------------------------------------------------------------------------------
Document2:

tunnels, containers of various types). 
 
Radioactive isotope:  An atom that gives off nuclear radiation and has the same number of protons (atomic number) as 
another atom but a different number of neutrons. 
 
Recy